In [24]:
from route_shape_process_scripts import *
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import numpy as np
import time

# import gtfs from 01_gtfs_transform notebook output

In [148]:
full_routes_gtfs = pd.read_csv("input_gtfs/gtfs_routes_2018-08-15_2018-12-12.csv")
full_shapes_gtfs = pd.read_csv("input_gtfs/gtfs_shapes_2018-08-15_2018-12-12.csv")
full_trips_gtfs = pd.read_csv("input_gtfs/gtfs_trips_2018-08-15_2018-12-12.csv")
full_trip_stop_schedule = pd.read_csv("input_gtfs/gtfs_2018-08-15_2018-12-12.csv")

/Users/benjaminmalnor/anaconda2/envs/geopy36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
full_trip_stop_schedule_dict = {}
for name, group in full_trip_stop_schedule.groupby(['start_gtfs_date','end_gtfs_date']):
    full_trip_stop_schedule_dict[name] = group

# analyze September --> November

## get route name -- id dictionary

In [4]:
route_name_to_id_dict = dict(zip(full_routes_gtfs.route_short_name.tolist(),full_routes_gtfs.route_id.tolist()))

In [80]:
routes_to_remove = ['South Lake Union Streetcar',
                   'First Hill Streetcar',
                   'Redmond LOOP',
                   'Trailhead Direct Issaquah Alps',
                   'Trailhead Direct Mt. Si',
                  'Trailhead Direct Mailbox Peak',
                  'Link light rail' ]
for routes in routes_to_remove:
    del route_name_to_id_dict[routes]

## select a route

In [82]:
#get_select_routeid_name(full_routes_gtfs, ['E Line'])[2]
route_of_interest = '120'
route_of_interest_id = route_name_to_id_dict[route_of_interest]
input_dict = {'route_id':route_of_interest_id}
input_dict

{'route_id': 100019}

# get subset of position files for just one route_id

In [84]:
month_list = ['201809', '201810', '201811']
full_route_positions = get_positions_months_route_id(month_list, input_dict['route_id'])

201809 9263521 Index(['route_id', 'timestamp', 'trip_id', 'vehicle_id', 'vehicle_lat',
       'vehicle_long', 'time_utc', 'time_pct'],
      dtype='object')
201810 10655967 Index(['route_id', 'timestamp', 'trip_id', 'vehicle_id', 'vehicle_lat',
       'vehicle_long', 'time_utc', 'time_pct'],
      dtype='object')
201811 10175465 Index(['route_id', 'timestamp', 'trip_id', 'vehicle_id', 'vehicle_lat',
       'vehicle_long', 'time_utc', 'time_pct'],
      dtype='object')


# add time index columns

In [85]:
full_route_positions = convert_index_to_pct(full_route_positions)
full_route_positions = add_time_index_columns(full_route_positions)

# need to print out shape_ids - find the shape to process
***
# TODO - get direction id get edges for all shapes but keep shape_id column for filtering

In [86]:
full_df = pd.DataFrame()
for name, group in full_trip_stop_schedule[full_trip_stop_schedule['route_id'] == input_dict['route_id']].groupby(['start_gtfs_date','end_gtfs_date']):
    #print(name)
    temp_df = group.groupby(['shape_id', 'direction_id','trip_headsign']).agg({'shape_id':'count'})\
                            .rename(columns={'shape_id':'shape_id_count'})\
                            .reset_index()
    #print(temp_df)
    if full_df.empty:
        full_df = temp_df
    else:
        full_df = full_df.append(temp_df)

In [87]:
full_df_sorted = full_df.groupby(['shape_id','direction_id','trip_headsign'])\
                .agg({'shape_id_count':'sum'})\
                .reset_index()\
                .sort_values('shape_id_count', ascending=False)

In [116]:
input_dict['direction_id']= 1

In [117]:
input_dict['shape_id'] = full_df_sorted.loc[full_df_sorted['direction_id']==input_dict['direction_id']].iloc[0]['shape_id']
trip_headsign = full_df_sorted.loc[full_df_sorted['direction_id']==direction_id].iloc[0]['trip_headsign']

# make route_vertex_geo_dict and G_dict

In [118]:
route_vertex_geo_dict = {}
G_dict = {}
for name, group in full_shapes_gtfs.groupby(
                ['start_gtfs_date','end_gtfs_date']):
    print(name)
    route_vertex_geo_dict[name], G_dict[name] = get_route_vertex_graph(group, input_dict['shape_id'])

('2018-08-15', '2018-09-23')
('2018-09-24', '2018-09-24')
('2018-09-25', '2018-11-01')
('2018-11-02', '2018-11-09')
('2018-11-10', '2018-12-11')
('2018-12-12', '2019-01-07')


# join position table with trip gtfs information

In [119]:
positions_w_trips = {}
for name, group in full_trips_gtfs.groupby(['start_gtfs_date','end_gtfs_date']):
    print(name)
    positions_w_trips[name] = join_positions_with_gtfs_trips(full_route_positions, group, name[0], name[1])

('2018-08-15', '2018-09-23')
('2018-09-24', '2018-09-24')
('2018-09-25', '2018-11-01')
('2018-11-02', '2018-11-09')
('2018-11-10', '2018-12-11')
('2018-12-12', '2019-01-07')


# get unique_trip_list_dict and positions_w_trips_geo_dict

In [120]:
unique_trip_list_dict = {}
positions_w_trips_geo_dict = {}

for gtfs_groups in positions_w_trips.keys():
    print(gtfs_groups)
    unique_trip_list_dict[gtfs_groups], positions_w_trips_geo_dict[gtfs_groups] = get_trip_from_shape_id(input_dict['shape_id'], 
                                                                                                        positions_w_trips[gtfs_groups])


('2018-08-15', '2018-09-23')
no trips
('2018-09-24', '2018-09-24')
no trips
('2018-09-25', '2018-11-01')
no trips
('2018-11-02', '2018-11-09')
('2018-11-10', '2018-12-11')
('2018-12-12', '2019-01-07')
no trips


# try parallel below - I don't think the parallel function will work on a Windows computer (I've had trouble in the past). It should work fine on a Mac.

In [121]:
import multiprocessing

In [122]:
def get_close_node_parallel(df_group, route_vertex_geo):
    '''
    '''
    df_group['close_node_tuple'] = df_group.apply(get_point_coords, route_vertex_geo=route_vertex_geo, axis=1).copy()
    return df_group

def get_point_coords(row, route_vertex_geo):
    if isinstance(row['geometry'], float):
        return ''
    else:
        return get_close_node(row['geometry'].coords[:][0], route_vertex_geo)

In [123]:
def full_step_process():
    n_pools = multiprocessing.cpu_count() - 1
    pool = multiprocessing.Pool(n_pools)
    positions_w_near_node_dict = {}
    for gtfs_group in positions_w_trips.keys():
        print("starting {}".format(gtfs_group))
        if positions_w_trips_geo_dict[gtfs_group].empty:
            pass
        else:
            grouped = positions_w_trips_geo_dict[gtfs_group].groupby('month_day_trip_veh')
            route_vertex_geo = route_vertex_geo_dict[gtfs_group]
            trip_group_tuple_list = []
            for name, group in grouped:
                trip_group_tuple_list.append((group, route_vertex_geo))
            positions_w_near_node_df = pd.concat(pool.starmap(get_close_node_parallel, trip_group_tuple_list))
            positions_w_near_node_dict[gtfs_group] = positions_w_near_node_df
    return positions_w_near_node_dict

# the below takes ~2-15 minutes on my computer depending on the shape/number of trips

In [124]:
if __name__ == "__main__":
    start = time.time()
    positions_w_near_node_dict = full_step_process()
    end = time.time()
    print(end - start)

starting ('2018-08-15', '2018-09-23')
starting ('2018-09-24', '2018-09-24')
starting ('2018-09-25', '2018-11-01')
starting ('2018-11-02', '2018-11-09')
starting ('2018-11-10', '2018-12-11')
starting ('2018-12-12', '2019-01-07')
226.91356778144836


In [125]:
def unpack_near_node_column(positions_w_near_node_dict):
    for gtfs_group in positions_w_near_node_dict.keys():
        positions_w_near_node_dict[gtfs_group]['near_node_pt'] = positions_w_near_node_dict[gtfs_group].apply(lambda x: x['close_node_tuple'][0], axis=1)
        positions_w_near_node_dict[gtfs_group]['shape_pt_sequence'] = positions_w_near_node_dict[gtfs_group].apply(lambda x: x['close_node_tuple'][1], axis=1)
        positions_w_near_node_dict[gtfs_group]['dist_to_nearest_route_pt'] = positions_w_near_node_dict[gtfs_group].apply(lambda x: x['close_node_tuple'][2], axis=1)
        positions_w_near_node_dict[gtfs_group].drop('close_node_tuple', axis=1, inplace=True)
        positions_w_near_node_dict[gtfs_group].drop_duplicates(['month_day_trip_veh','shape_pt_sequence'], keep='last', inplace=True)
    return positions_w_near_node_dict

In [126]:
unpacked_positions_w_near_node_dict = unpack_near_node_column(positions_w_near_node_dict)

# merge all dictionaries into one

In [127]:
for idx, dict_group in enumerate(unpacked_positions_w_near_node_dict.keys()):
    print(dict_group)
    if idx == 0:
        unpacked_positions_full = unpacked_positions_w_near_node_dict[dict_group].copy()
    else:
        unpacked_positions_full = unpacked_positions_full.append(unpacked_positions_w_near_node_dict[dict_group])

('2018-11-02', '2018-11-09')
('2018-11-10', '2018-12-11')


# export

In [146]:
unpacked_positions_full.to_csv('transformed/route_{}_{}_shape_{}_raw_w_nearest_2018-08-15_2018-12-11.csv'.format(
                                        route_of_interest,"".join(trip_headsign.split(" ")) ,input_dict['shape_id']), index=False)

# get start of schedue time (join in gtfs) then calculate just timing to shape distance

In [129]:
def datetime_transform_df(df):
    '''
    '''
    df['time_pct'] = df['time_pct'].apply(pd.to_datetime)
    df.set_index('time_pct', inplace=True)
    df.sort_index(inplace=True)
    df = df.tz_localize('UTC')
    df = df.tz_convert('US/Pacific')
    
    return df

In [130]:
def join_tripstart_unpacked(unpacked_positions_w_near_node_df, trip_stops_w_name_route):
    '''
    '''
    normal_hours = [6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    #drop hours near '0' because the hour comparison gets messed up - gtfs has hours > 23 :(
    unpacked_positions_w_near_node_df = unpacked_positions_w_near_node_df[unpacked_positions_w_near_node_df.hour.isin(normal_hours)]
    
    if 'trip_stops_w_name_route' in trip_stops_w_name_route.columns:
        trip_stops_w_name_route.drop('trip_stops_w_name_route', axis=1, inplace=True)
    position_w_node_schedule = unpacked_positions_w_near_node_df.merge(trip_stops_w_name_route,how='left',
                                                    left_on=['trip_id','route_id','shape_id','shape_pt_sequence'], 
                                                    right_on=['trip_id','route_id','shape_id','stop_sequence'])
    
    position_w_node_schedule = position_w_node_schedule[position_w_node_schedule['stop_name'].notnull()]
    
    
    
    position_w_node_schedule = datetime_transform_df(position_w_node_schedule)
    
    position_w_node_schedule.drop_duplicates(['month_day_trip_veh','shape_pt_sequence'], keep='last', inplace=True)

    position_w_node_schedule['trip_start_time'] = position_w_node_schedule['trip_start_time'].apply(pd.to_datetime)
        
    #take the time at every stop and subtract the scheduled start time
    position_w_node_schedule['time_from_scheduled_start'] = (((position_w_node_schedule.index.hour)*60+
                                                    position_w_node_schedule.index.minute+
                                                    (position_w_node_schedule.index.second)/60) - 
                                                    ((position_w_node_schedule.loc[:,'trip_start_time'].dt.hour)*60+
                                                    position_w_node_schedule.loc[:,'trip_start_time'].dt.minute+
                                                    (position_w_node_schedule.loc[:,'trip_start_time'].dt.second)/60))

    return position_w_node_schedule

# could be done in parallel!

In [131]:
start = time.time()
unpacked_positions_full_w_start = join_tripstart_unpacked(unpacked_positions_full, full_trip_stop_schedule)
end = time.time()
print(end - start)

18.61123776435852


# export

In [147]:
unpacked_positions_full_w_start.to_csv('transformed/route_{}_{}_shape_{}_stopsonly_2018-08-15_2018-12-11.csv'.format(
                                        route_of_interest,"".join(trip_headsign.split(" ")), input_dict['shape_id']), index=False)